In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./online_retail_clean.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceYearMonth'] = df['InvoiceDate'].dt.to_period('M')


In [ ]:
# 1. 각 고객의 첫 구매 월 (코호트)
first_purchase = df.groupby('CustomerID')['InvoiceYearMonth'].min().reset_index()
first_purchase.columns = ['CustomerID', 'CohortMonth']

print(f"\n고객별 첫 구매 월 산출 완료")
print(f"코호트 기간: {first_purchase['CohortMonth'].min()} ~ {first_purchase['CohortMonth'].max()}")



고객별 첫 구매 월 산출 완료
코호트 기간: 2010-12 ~ 2011-12


In [4]:
# 2. 원본 데이터에 코호트 정보 추가
df = df.merge(first_purchase, on='CustomerID', how='left')
print(f"\n원본 데이터에 코호트 정보 추가 완료")
print(df.head())


원본 데이터에 코호트 정보 추가 완료
   InvoiceNo StockCode                          Description  Quantity  \
0     536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1     536365     71053                  WHITE METAL LANTERN         6   
2     536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3     536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4     536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country InvoiceYearMonth  \
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom          2010-12   
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom          2010-12   
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom          2010-12   
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom          2010-12   
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom          2010-12   

  InvoiceDate_only  Revenue CohortMonth  
0       20

In [5]:
# 3. 코호트 인덱스 계산 (첫 구매 이후 몇 개월차)
def get_month_diff(row):
    return (row['InvoiceYearMonth'].to_timestamp().year - row['CohortMonth'].to_timestamp().year) * 12 + \
           (row['InvoiceYearMonth'].to_timestamp().month - row['CohortMonth'].to_timestamp().month)

df['CohortIndex'] = df.apply(get_month_diff, axis=1)

print(f"코호트 인덱스 계산 완료 (0개월 = 첫 구매 월)")


코호트 인덱스 계산 완료 (0개월 = 첫 구매 월)


In [6]:
# 4. 코호트별 고객 수 (코호트 크기)
cohort_size = df.groupby('CohortMonth')['CustomerID'].nunique().reset_index()
cohort_size.columns = ['CohortMonth', 'CohortSize']

print(f"\n코호트 크기:")
print(cohort_size.head(10))



코호트 크기:
  CohortMonth  CohortSize
0     2010-12         885
1     2011-01         417
2     2011-02         380
3     2011-03         452
4     2011-04         300
5     2011-05         284
6     2011-06         242
7     2011-07         188
8     2011-08         169
9     2011-09         299


In [7]:
# 5. 코호트별, 월별 활성 고객 수
cohort_data = df.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].nunique().reset_index()
cohort_data.columns = ['CohortMonth', 'CohortIndex', 'ActiveCustomers']
print(f"\n코호트별, 월별 활성 고객 수:")
print(cohort_data.head(10))


코호트별, 월별 활성 고객 수:
  CohortMonth  CohortIndex  ActiveCustomers
0     2010-12            0              885
1     2010-12            1              324
2     2010-12            2              286
3     2010-12            3              340
4     2010-12            4              321
5     2010-12            5              352
6     2010-12            6              321
7     2010-12            7              309
8     2010-12            8              313
9     2010-12            9              350


In [8]:
# 6. 코호트 크기 추가
cohort_data = cohort_data.merge(cohort_size, on='CohortMonth', how='left')
print(f"\n코호트 크기 추가 완료")
print(cohort_data.head(10))


코호트 크기 추가 완료
  CohortMonth  CohortIndex  ActiveCustomers  CohortSize
0     2010-12            0              885         885
1     2010-12            1              324         885
2     2010-12            2              286         885
3     2010-12            3              340         885
4     2010-12            4              321         885
5     2010-12            5              352         885
6     2010-12            6              321         885
7     2010-12            7              309         885
8     2010-12            8              313         885
9     2010-12            9              350         885


In [9]:
# 7. Retention Rate 계산
cohort_data['RetentionRate'] = (cohort_data['ActiveCustomers'] / cohort_data['CohortSize'] * 100).round(2)

print(f"\nRetention Rate 계산 완료")


Retention Rate 계산 완료


In [10]:
# 8. 피벗 테이블 생성 (시각화용)
cohort_pivot = cohort_data.pivot_table(
    index='CohortMonth',
    columns='CohortIndex',
    values='RetentionRate'
)

print(f"\n코호트 피벗 테이블 (Retention Rate %):")
print(cohort_pivot)
print("\n코호트 분석 완료!")


코호트 피벗 테이블 (Retention Rate %):
CohortIndex     0      1      2      3      4      5      6      7      8   \
CohortMonth                                                                  
2010-12      100.0  36.61  32.32  38.42  36.27  39.77  36.27  34.92  35.37   
2011-01      100.0  22.06  26.62  23.02  32.13  28.78  24.70  24.22  29.98   
2011-02      100.0  18.68  18.68  28.42  27.11  24.74  25.26  27.89  24.74   
2011-03      100.0  15.04  25.22  19.91  22.35  16.81  26.77  23.01  27.88   
2011-04      100.0  21.33  20.33  21.00  19.67  22.67  21.67  26.00   7.33   
2011-05      100.0  19.01  17.25  17.25  20.77  23.24  26.41   9.51    NaN   
2011-06      100.0  17.36  15.70  26.45  23.14  33.47   9.50    NaN    NaN   
2011-07      100.0  18.09  20.74  22.34  27.13  11.17    NaN    NaN    NaN   
2011-08      100.0  20.71  24.85  24.26  12.43    NaN    NaN    NaN    NaN   
2011-09      100.0  23.41  30.10  11.37    NaN    NaN    NaN    NaN    NaN   
2011-10      100.0  24.02  11.45

In [11]:
# 저장
cohort_data.to_csv('./cohort_analysis.csv', index=False)
cohort_pivot.to_csv('./cohort_pivot.csv')
print(f"\n✅ Cohort 분석 완료")
print(f"   - cohort_analysis.csv (상세 데이터)")
print(f"   - cohort_pivot.csv (피벗 테이블)")


✅ Cohort 분석 완료
   - cohort_analysis.csv (상세 데이터)
   - cohort_pivot.csv (피벗 테이블)


# 핵심 발견
**2010년 12월 코호트 (885명 - 최대 규모)**
- 1개월 후 재구매율: 36.61%   
- 2개월 후: 32.32%   
- 3개월 후: 38.42%   
- 12개월 후: 26.55%